# Imports

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import pytz
from datetime import datetime as dt
from datetime import timedelta as td
from hbt_tools import utils_hbt as uh
from hbt_tools import plots as hp
from pathlib import Path

# Functions

In [ ]:
def plot_pw(ts_pw, ts_rain, names=['PW', 'rain']):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=ts_pw.index,
        y=ts_pw.iloc[:,0],
        name=names[0],
        line=dict(color='red', width=1),
        yaxis="y1"
        ))
    
    fig.add_trace(go.Bar(
        x=ts_rain.index,
        y=ts_rain.iloc[:,0],
        name=names[1],
        marker_color='blue', 
        yaxis="y2"
        ))

    fig.update_layout(
        yaxis=dict(
            title="Niveau",
            titlefont=dict(color="red"),
            tickfont=dict(color="red"),
            anchor="free",
            side="left",
        ),
        yaxis2=dict(
            title="Regenmenge [mm/10 min]",
            titlefont=dict(color="blue"),
            tickfont=dict(color="blue"),
            anchor="free",
            overlaying="y",
            side="right",
        )
    )

    return fig
#-------------------------------------------------------------------------------

def filter_ts(ts, date_from, date_to):
    ts = ts[(ts.index>=date_from) & (ts.index<date_to)]

    return ts
#-------------------------------------------------------------------------------

def del_values(ts, val):
    mask = ts.iloc[:,0]==val
    ts_out = ts.drop(ts[mask].index)
    return ts_out
#-------------------------------------------------------------------------------

def ts_delta_t(ts):
    '''
    ts: timeseries
    return: delta_t
    '''
    ts_out = pd.DataFrame()
    ts_out['delta_t'] = ts.index
    delta_t = ts_out['delta_t'] - ts_out['delta_t'].shift()

    ts_out['delta_t'] = delta_t

    return ts_out
#-------------------------------------------------------------------------------

def poleni(ts_h, B, mu=0.5):
    '''
    h: ts [m]
    B: Breite [m]
    return: Q_pol [m3/s]
    '''
    ts_h['Q [m3/s]'] = 2/3 * mu * (2 * 9.81)**(1/2) * B * (ts_h.iloc[:,0])**(3/2)
    ts_h['delta_t'] = ts_h.index
    ts_h['delta_t'] = (ts_h['delta_t'] - ts_h['delta_t'].shift()).dt.total_seconds()
    ts_h['volume'] = ts_h['Q [m3/s]'] * ts_h['delta_t']
    
    return ts_h
#-------------------------------------------------------------------------------

def add_ply_hline(fig, y, yref='y2', color='red', width=0.8):
    fig.add_shape(type="line", 
        xref="paper", yref=yref,
        x0=0, y0=y, x1=0.94, y1=y,
        line=dict(
            color=color,
            width=width
        ),
    )
#-------------------------------------------------------------------------------

# Parameters

In [ ]:
dt_format = '%d.%m.%Y %H:%M'
save_dir = Path(r"H:\2 Projekte\8000-\8500er\8596\8596.36-Mühlestrasse"
    r"\05 Berechnungen Grundlagen\Datenauswertung\Plots")
date_from = dt(2021, 10, 1, tzinfo=pytz.timezone('Europe/Zurich'))
date_to = dt(2023, 10, 1, tzinfo=pytz.timezone('Europe/Zurich'))

# Rain Data

In [ ]:
id = 2039
date_format = '%d.%m.%Y %H:%M'
rain = uh.hbtdb_get_data(id, date_from, date_to, date_format)

# Pumpwerke

## PW Sonnenbergstrasse

In [ ]:
# Read data
id = 457
lvl_sonnenberg = uh.hbtdb_get_data(id, date_from=date_from, date_to=date_to)

In [ ]:
h_messband = 4.67 # m
h_drucksonde = 0.35 # m
lvl_kante_pw = 469.66 # m ü.M.
lvl_ül = 468.20 # m ü.M.
lvl_min = 464.90 # m ü.M.

h_corr = lvl_kante_pw - h_messband - h_drucksonde

lvl_sonnenberg += h_corr

In [ ]:
fig = hp.ply_2y(lvl_sonnenberg, rain, names=['Niveau PW', 'Regen LUZ'],
    xlabel='Zeit', ylabels=['Niveau [m ü.M.]', 'Regen [mm/10min]'],
    colors=['red', 'blue'])

In [ ]:
fig.update_layout(
    title='PW Sonnenbergstrasse', font=dict(size=25)
)

fig.add_hline(y=lvl_ül, line_width=1, line_color='black')
fig.add_annotation(x=lvl_sonnenberg.index[100000], y=lvl_ül, text=f'Überlauf {lvl_ül} m ü.M.',
    font=dict(size=20, color='black'))

In [ ]:
save_path = save_dir / 'Niveau_PW-Sonnenbergstrasse.html'
fig.write_html(save_path)

### Entlastungen

In [ ]:
# Entlastungen
entl_sonnenberg = lvl_sonnenberg - lvl_ül
mask = entl_sonnenberg['wert'] < 0
entl_sonnenberg.loc[mask, 'wert'] = 0
entl_sonnenberg = poleni(entl_sonnenberg, B=2.6)
tot_entl = entl_sonnenberg['volume'].sum()
print(f'Tot. Entlastungsmenge Sonnenberg: {tot_entl:0.0f} m3')

In [ ]:
save_path = Path(r"H:\2 Projekte\8000-\8500er\8596\8596.36-Mühlestrasse"
    r"\05 Berechnungen Grundlagen\Datenauswertung\Hydraulischer Wirkungsgrad\ent_sonnenberg.csv")
mask = entl_sonnenberg['volume']!=0
entl_sonnenberg.loc[mask, :].to_csv(save_path, sep=';')

## PW Hirsen

In [ ]:
# Read data
id = 432
lvl_hirsen = uh.hbtdb_get_data(id, date_from=date_from, date_to=date_to)
lvl_hirsen = filter_ts(lvl_hirsen, date_from, date_to)
lvl_hirsen = lvl_hirsen / 100 # cm -> m

In [ ]:
h_messband = 4.73 # m
h_drucksonde = 0.94 # m
lvl_kante_pw = 446.30 # m ü.M.
lvl_ül = 443.80 # m ü.M.
lvl_min = 440.70 # m ü.M.

h_corr = lvl_kante_pw - h_messband - h_drucksonde

lvl_hirsen += h_corr

In [ ]:
fig = hp.ply_2y(lvl_hirsen, rain, names=['Niveau PW', 'Regen LUZ'],
    xlabel='Zeit', ylabels=['Niveau [m ü.M.]', 'Regen [mm/10min]'],
    colors=['red', 'blue']);

In [ ]:
fig.update_layout(
    title='PW Hirsen', font=dict(size=25)
)

fig.add_hline(y=lvl_ül, line_width=1, line_color='black')
fig.add_annotation(x=lvl_hirsen.index[100000], y=lvl_ül,text=f'Überlauf {lvl_ül} m ü.M.',
    font=dict(size=20, color='black'))

In [ ]:
save_path = save_dir / 'Niveau_PW-Hirsen.html'
fig.write_html(save_path)

### Entlastungen

In [ ]:
entl_hirsen = lvl_hirsen - lvl_ül
mask = entl_hirsen['wert'] < 0
entl_hirsen.loc[mask, 'wert'] = 0
entl_hirsen = poleni(entl_hirsen, B=0.5)
tot_entl = entl_hirsen['volume'].sum()
print(f'Tot. Entlastungsmenge Hirsen: {tot_entl:0.0f} m3')

In [ ]:
save_path = Path(r"H:\2 Projekte\8000-\8500er\8596\8596.36-Mühlestrasse"
    r"\05 Berechnungen Grundlagen\Datenauswertung\Hydraulischer Wirkungsgrad\ent_hirsern2023.csv")
mask = entl_hirsen['volume']!=0
entl_hirsen.loc[mask, :].to_csv(save_path, sep=';')

## PW Sonnenbergstrasse & PW Hirsen

In [ ]:

fig = hp.ply_2y(lvl_sonnenberg, lvl_hirsen, names=['Niveau PW Sonnenberg',
    'Niveau PW Hirsen'], colors=['green', 'magenta'], xlabel='Zeit',
    ylabels=['Niveau [m ü.M.]', 'Niveau [m ü.M.]']);

In [ ]:
fig.update_layout(
    title='PW Hirsen und PW Sonnenberg',
    font=dict(size=25),
    plot_bgcolor='white'
)

fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)

lvl_ül_sonnenberg = 468.20 # m ü.M.
lvl_ül_hirsen = 443.80 # m ü.M.
p1_hirsen = 442.35 # m ü.M.
p2_hirsen = 443.30 # m ü.M.
aus_hirsen = 440.70 # m ü.M.
p1_sonnenberg = 466.60 # m ü.M.
p2_sonnenberg = 468.00 # m ü.M.
aus_sonnenberg = 464.90 # m ü.M.

fig.add_hline(y=lvl_ül_sonnenberg, line_width=0.8, line_color='black')

add_ply_hline(fig, y=lvl_ül_hirsen, color='black')

In [ ]:
save_path = save_dir / 'Niveau_PW-Hirsen-Sonnenbergstrsase2023.html'
fig.write_html(save_path)

# SABA

## ASB

### Niveau Einlaufkammer

In [ ]:
id = 458
asb1 = uh.hbtdb_get_data(id, date_from=date_from, date_to=date_to)

In [ ]:
fig = hp.ply_2y(asb1, rain, names=['Niveau Einlauf ASB', 'Regen LUZ'],
    xlabel='Zeit', ylabels=['Niveau [cm]', 'Regen [mm/10min]'],
    colors=['red', 'blue'])

In [ ]:
save_path = save_dir / 'Niveau_Einlauf-ASB.html'
fig.write_html(save_path)

### Niveau Becken

In [ ]:
id = 465

# Niveau
asb2 = uh.hbtdb_get_data(id, date_from=date_from, date_to=date_to)

In [ ]:
# Correct 'asb2'
asb2 = asb2 - 105 # cm, 1.05 Messwert bei ansprigngen des Wehres/Entlastung (Christian)
asb2 = asb2 + 0.9

# Durchfluss
B = 4 # der Wert ist aus dem Schalungsplan
mu = 0.6
q_asb = poleni(B=B, mu=mu, ts_h=asb2/100) # l/s
q_asb.fillna(0, inplace=True)

In [ ]:
q_asb

In [ ]:
fig = hp.ply_2y(asb2, rain, names=['Niveau ASB', 'Regen LUZ'], 
    xlabel='Zeit', ylabels=['Niveau [cm]', 'Regen [mm/10min]'],
    colors=['red', 'blue'])

In [ ]:
fig.update_layout(
    title='Absetzbecken', font=dict(size=25)
)

fig.update_xaxes(title=dict(font=dict(size=25)))
fig.update_yaxes(title=dict(font=dict(size=25)));

In [ ]:
save_path = save_dir / 'Niveau_ASB.html'
fig.write_html(save_path)

### Abfluss ASB

In [ ]:
fig = hp.ply_2y(asb2, q_asb, names=['Niveau ASB', 'Abfluss ASB'], 
    xlabel='Zeit', ylabels=['Niveau [cm]', 'Durchfluss [l/s]'],
    colors=['red', 'blue'])

In [ ]:
fig.update_layout(
    title='Abfluss Absetzbecken', font=dict(size=25)
)

fig.update_xaxes(title=dict(font=dict(size=25)))
fig.update_yaxes(title=dict(font=dict(size=25)));

In [ ]:
save_path = save_dir / 'Abfluss_ASB.html'
fig.write_html(save_path)

## Ablauf SABA

In [ ]:
id = 466
q_ab = uh.hbtdb_get_data(id, date_from=date_from, date_to=date_to)

In [ ]:
fig = hp.ply_2y(q_ab, rain, names=['Ablauf SABA', 'Regen LUZ'], xlabel='Zeit',
    ylabels=['Durchfluss [l/s]', 'Regen [mm/10min]'], colors=['red', 'blue'])

In [ ]:
fig.update_layout(
    title='SABA Ablauf', font=dict(size=25)
)

fig.update_xaxes(title=dict(font=dict(size=25)))
fig.update_yaxes(title=dict(font=dict(size=25)));

In [ ]:
save_path = save_dir / 'Ablauf_SABA2023.html'
fig.write_html(save_path)

In [ ]:
delta_t = ts_delta_t(q_ab)
delta_t.sort_values(by=delta_t.columns[0], ascending=False, inplace=True)
# delta_t = delta_t[delta_t>pd.Timedelta(minutes=1)]

save_path = Path(r"H:\2 Projekte\8000-\8500er\8596\8596.36-Mühlestrasse"
    r"\05 Berechnungen Grundlagen\Datenauswertung")
save_path = save_path / 'Delta-t_Q-ab2023.csv'
delta_t.to_csv(save_path, sep=';')

### Entlastungen

In [ ]:
q_entl = q_ab.copy()
q_entl['bool'] = 0
date_entl = [
    [dt(2022, 6, 24, 12, 35), dt(2022, 6, 24, 21, 54)],
    [dt(2022, 6, 30, 20, 27), dt(2022, 7, 1, 20, 21)],
    [dt(2022, 8, 19, 0, 49), dt(2022, 8, 19, 22, 19)],
    [dt(2022, 9, 8, 2, 10), dt(2022, 9, 8, 11, 34)],
    [dt(2022, 9, 15, 18, 30), dt(2022, 9, 16, 17, 54)],
    [dt(2022, 9, 28, 6, 47), dt(2022, 9, 29, 4, 30)],
    [dt(2022, 12, 21, 21, 00), dt(2023, 1, 2, 1, 00)],
    [dt(2023, 3, 13, 18, 0), dt(2023, 3, 14, 9, 0)],
    [dt(2023, 4, 29, 15, 30), dt(2023, 4, 29, 21, 30)],
]

for i in range(len(date_entl)):
    date_entl[i][0] = date_entl[i][0].astimezone(pytz.timezone('Europe/Zurich'))
    date_entl[i][1] = date_entl[i][1].astimezone(pytz.timezone('Europe/Zurich'))

# Select Entlastungsereignisse
i = 1
for date in date_entl:
    mask = ((q_entl.index > date[0]) & (q_entl.index <= date[1]))
    q_entl.loc[mask, 'bool'] = i
    i+=1

q_entl = q_entl[q_entl['bool'] != 0]

# Set Anspringsdurchfluss
q_anspr = [39, 10, 28, 22, 26, 18, 3, 7.5, 6.5]
for i in range(len(q_anspr)):
    mask = (q_entl['bool'] == i)
    q_entl.loc[mask, 'wert'] -= q_anspr[i-1]
    mask = q_entl['wert'] < 0
    q_entl.loc[mask, 'wert'] = 0

In [ ]:
fig = hp.ply_1y(q_entl, xlabel='Zeit', names=['Ablauf SABA'], ylabel='Durchfluss [l/s]')

In [ ]:
fig.update_layout(
    title='Entlastungsdurchfluss SABA', font=dict(size=25)
)

fig.update_xaxes(title=dict(font=dict(size=25)))
fig.update_yaxes(title=dict(font=dict(size=25)));

In [ ]:
save_path = save_dir / 'Entlastungsdurchfluss_SABA_2023.html'
fig.write_html(save_path)

In [ ]:
# Berechnung Gesamt-Entlastungsvolumen
q_entl['delta_t'] = q_entl.index
q_entl['delta_t'] = q_entl['delta_t'] - q_entl['delta_t'].shift()
q_entl.iloc[0, 2] = td(0)

q_entl['volume [l]'] = q_entl['wert'] * q_entl['delta_t'].dt.total_seconds()

tot_vol = q_entl['volume [l]'].sum() / 1e3
print(f"Gesamt-Entlastungsvolumen: {tot_vol:0.0f} m3")

In [ ]:
save_path = Path(r"H:\2 Projekte\8000-\8500er\8596\8596.36-Mühlestrasse"
    r"\05 Berechnungen Grundlagen\Datenauswertung\Hydraulischer Wirkungsgrad\ent_saba.csv")
mask = q_entl['wert'] != 0
q_entl = q_entl[mask]
q_entl.to_csv(save_path, sep=';')

## RFB

### RFB 1

In [ ]:
id = 459
rfb1 = uh.hbtdb_get_data(id, date_from=date_from, date_to=date_to)

In [ ]:
# Reframe data points
a = 1.74
rfb1 = (rfb1 - a) * -1 * 100
rfb1 = del_values(rfb1, 74)

In [ ]:
id = 460
rfb1_d_ = uh.hbtdb_get_data(id, date_from=date_from, date_to=date_to)

In [ ]:
# Reframe data points
rfb1_d = rfb1_d_ * 100

In [ ]:
fig = hp.ply_2y(rfb1, rain, names=['Niveau RFB 1', 'Regen LUZ'],
    xlabel='Zeit', ylabels=['Niveau [cm]', 'Regen [mm/10min]'],
    colors=['red', 'blue'])

In [ ]:
fig.update_layout(
    title='Niveau RFB 1', font=dict(size=25)
)

fig.update_xaxes(title=dict(font=dict(size=25)))
fig.update_yaxes(title=dict(font=dict(size=25)))

h_ül = 66  # cm - gemäss Fotos Begehung
fig.add_hline(y=h_ül, line_width=1, line_color='black')
fig.add_annotation(x=rfb1.index[0], y=h_ül,text=f'{h_ül} cm',
    font=dict(size=20, color='black'))
fig

In [ ]:
save_path = save_dir / 'Niveau_RFB1_2023.html'
fig.write_html(save_path)

In [ ]:
fig = hp.ply_2y([rfb1, rfb1_d], q_ab,
    names=['Niveau Radar', 'Niveau Drucksonde', 'Ablauf'],
    xlabel='Zeit', ylabels=['Niveau [cm]', 'Abfluss [l/s]'],
    colors=['blue', 'red', 'green'])

In [ ]:
fig.update_layout(
    title='Niveau RFB 1', font=dict(size=25)
);

In [ ]:
save_path = save_dir / 'Niveau_Druck_RFB1_2023.html'
fig.write_html(save_path)

### RFB 2

In [ ]:
id = 461
rfb2 = uh.hbtdb_get_data(id, date_from=date_from, date_to=date_to)

In [ ]:
# Reframe data points
rfb2 = (rfb2 - 0.92) * -1 * 100
rfb2 = del_values(rfb2, 92)

In [ ]:
fig = hp.ply_2y(rfb2, rain, names=['Niveau RFB 2', 'Regen LUZ'],
    xlabel='Zeit', ylabels=['Niveau [cm]', 'Regen [mm/10min]'],
    colors=['red', 'blue'])

In [ ]:
fig.update_layout(
    title='Niveau RFB 2', font=dict(size=25)
)

fig.update_xaxes(title=dict(font=dict(size=25)))
fig.update_yaxes(title=dict(font=dict(size=25)))

h_ül = 62 # cm - gemäss Fotos Begehung
fig.add_hline(y=h_ül, line_width=1, line_color='black')
fig.add_annotation(x=rfb2.index[0], y=h_ül,text=f'{h_ül} cm',
    font=dict(size=20, color='black'))
fig

In [ ]:
save_path = save_dir / 'Niveau_RFB2_2023.html'
fig.write_html(save_path)

In [ ]:
fig = hp.ply_2y(rfb2, q_ab,
    names=['Niveau Radar', 'Ablauf'],
    xlabel='Zeit', ylabels=['Niveau [cm]', 'Abfluss [l/s]'],
    colors=['blue', 'green'])

In [ ]:
fig.update_layout(
    title='Niveau RFB 2', font=dict(size=25)
);

In [ ]:
save_path = save_dir / 'Niveau_Druck_RFB2_2023.html'
fig.write_html(save_path)

In [ ]:
fig = hp.ply_2y([rfb1, rfb2], q_ab,
    names=['Niveau RFB 1', 'Niveau RFB 2', 'Ablauf SABA'],
    xlabel='Zeit', ylabels=['Niveau [cm]', 'Abfluss [l/s]'],
)

In [ ]:
fig.update_layout(
    font=dict(size=25)
);

In [ ]:
save_path = save_dir / 'Ablauf-RFB1-RFB2_2023.html'
fig.write_html(save_path)